# Reference Paper
## An efficient multiple-classifier system for Arabic calligraphy style recognition

In [19]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from skimage.morphology import skeletonize
from sklearn.neighbors import KNeighborsClassifier
import os
from sklearn import metrics
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from __future__ import division
from scipy.signal import convolve2d
from sklearn.ensemble import VotingClassifier

In [2]:
## Reference: https://stackoverflow.com/questions/23548863/converting-a-specific-matlab-script-to-python/23575137
def lpq(img,winSize=3,freqestim=1,mode='nh'):
    rho=0.90

    STFTalpha=1/winSize  # alpha in STFT approaches (for Gaussian derivative alpha=1)
    sigmaS=(winSize-1)/4 # Sigma for STFT Gaussian window (applied if freqestim==2)
    sigmaA=8/(winSize-1) # Sigma for Gaussian derivative quadrature filters (applied if freqestim==3)

    convmode='valid' # Compute descriptor responses only on part that have full neigborhood. Use 'same' if all pixels are included (extrapolates np.image with zeros).

    img=np.float64(img) # Convert np.image to double
    r=(winSize-1)/2 # Get radius from window size
    x=np.arange(-r,r+1)[np.newaxis] # Form spatial coordinates in window

    if freqestim==1:  #  STFT uniform window
        #  Basic STFT filters
        w0=np.ones_like(x)
        w1=np.exp(-2*np.pi*x*STFTalpha*1j)
        w2=np.conj(w1)

    ## Run filters to compute the frequency response in the four points. Store np.real and np.imaginary parts separately
    # Run first filter
    filterResp1=convolve2d(convolve2d(img,w0.T,convmode),w1,convmode)
    filterResp2=convolve2d(convolve2d(img,w1.T,convmode),w0,convmode)
    filterResp3=convolve2d(convolve2d(img,w1.T,convmode),w1,convmode)
    filterResp4=convolve2d(convolve2d(img,w1.T,convmode),w2,convmode)

    # Initilize frequency domain matrix for four frequency coordinates (np.real and np.imaginary parts for each frequency).
    freqResp=np.dstack([filterResp1.real, filterResp1.imag,
                        filterResp2.real, filterResp2.imag,
                        filterResp3.real, filterResp3.imag,
                        filterResp4.real, filterResp4.imag])

    ## Perform quantization and compute LPQ codewords
    inds = np.arange(freqResp.shape[2])[np.newaxis,np.newaxis,:]
    LPQdesc=((freqResp>0)*(2**inds)).sum(2)

    ## Switch format to uint8 if LPQ code np.image is required as output
    if mode=='im':
        LPQdesc=np.uint8(LPQdesc)

    ## Histogram if needed
    if mode=='nh' or mode=='h':
        LPQdesc=np.histogram(LPQdesc.flatten(),range(256))[0]

    ## Normalize histogram if needed
    if mode=='nh':
        LPQdesc=LPQdesc/LPQdesc.sum()

    #print(LPQdesc)
    return LPQdesc

In [3]:
trainData = []
trainDataLabels = []

In [4]:
def preprocessing(image):
    ret2,th2 = cv2.threshold(image,0,1,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    return th2

In [5]:
def extractFeatures(image):
    return lpq(image, winSize=5, mode='nh')

In [6]:
fontFile = open("names.txt",'r')
fonts = np.loadtxt(fontFile, dtype='str')
for font in fonts:
    fontDir, fontName = font.split("___")
    print(fontDir, fontName)
    for file in os.listdir(fontDir):
        image = cv2.imread(fontDir+"/"+file,0)
        image_processed = preprocessing(image)
        trainData.append(image_processed)
        trainDataLabels.append(fontName)

1 diwani
2 naskh
3 parsi
4 rekaa
5 thuluth
6 maghribi
7 kufi
8 mohakek
9 Squar-kufic


In [7]:
trainData = np.asarray(trainData)
trainDataLabels = np.asarray(trainDataLabels)

N = trainData.shape[0]
trainFeatures = np.zeros((N, 255))

for i in range(trainFeatures.shape[0]):
    trainFeatures[i] = extractFeatures(trainData[i])
    
#print(trainData.shape, trainDataLabels.shape)

E:\hebaa\anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


## KNN

In [40]:
clf_knn = KNeighborsClassifier(n_neighbors=3)
clf_knn.fit(trainFeatures, trainDataLabels)
y_pred_knn = clf_knn.predict(trainFeatures)
print("Accuracy KNN:",metrics.accuracy_score(trainDataLabels, y_pred_knn)*100)

Accuracy KNN: 94.540059347181


## SVM

In [43]:
clf_svm = svm.SVC(decision_function_shape='ovo',probability=True, kernel='rbf')
clf_svm.fit(trainFeatures, trainDataLabels)
y_pred_svm = clf_svm.predict(trainFeatures)
print("Accuracy SVM:",metrics.accuracy_score(trainDataLabels, y_pred_svm)*100)

Accuracy SVM: 56.73590504451038


## MLPClassifier

In [51]:
clf_mlp = MLPClassifier(random_state=1, max_iter=500)
clf_mlp.fit(trainFeatures, trainDataLabels)
y_pred_mlp = clf_mlp.predict(trainFeatures)
print("Accuracy MLPClassifier:",metrics.accuracy_score(trainDataLabels, y_pred_mlp)*100)

Accuracy MLPClassifier: 98.57566765578635


E:\hebaa\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


## Combining Classifiers

###  Majority/Plurality Vote

In [49]:
clf_knn = KNeighborsClassifier(n_neighbors=5)
clf_mlp = MLPClassifier(random_state=1, max_iter=500)
clf_svm = svm.SVC(decision_function_shape='ovo',probability=True)


clf_max = VotingClassifier(estimators=[('knn', clf_knn), ('svm', clf_svm), ('mlp', clf_mlp)], voting='hard')
clf_max.fit(trainFeatures, trainDataLabels)
y_pred_max = clf_max.predict(trainFeatures)

print("Accuracy - Majority Vote:",metrics.accuracy_score(trainDataLabels, y_pred_max)*100)

E:\hebaa\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy - Majority Vote: 91.98813056379822


###  Sum

In [50]:
clf_knn = KNeighborsClassifier(n_neighbors=5)
clf_mlp = MLPClassifier(random_state=1, max_iter=500)
clf_svm = svm.SVC(decision_function_shape='ovo',probability=True)


clf_max = VotingClassifier(estimators=[('knn', clf_knn), ('svm', clf_svm), ('mlp', clf_mlp)], voting='soft')
clf_max.fit(trainFeatures, trainDataLabels)
y_pred_max = clf_max.predict(trainFeatures)

print("Accuracy - Sum:",metrics.accuracy_score(trainDataLabels, y_pred_max)*100)

E:\hebaa\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy - Sum: 95.78635014836794
